# Build a variational autoencoder

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg()[-3:] for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
test_strs = [g.get_reg()[-3:] for _ in range(n_test)]
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print(vec_length)

108


### Variational autoencoder

In [3]:
latent_dim = 100
intermediate_dim = 300
original_dim = vec_length

inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [4]:
vae.fit(train_vecs, train_vecs,
        epochs=50,
        batch_size=256,
        shuffle=True,
        validation_data=(test_vecs, test_vecs))

Epoch 1/50
40/40 [==============================] - 1s 34ms/step - loss: 62.4613 - val_loss: 49.0878
Epoch 2/50
40/40 [==============================] - 1s 23ms/step - loss: 42.4336 - val_loss: 37.2717
Epoch 3/50
40/40 [==============================] - 1s 22ms/step - loss: 34.4249 - val_loss: 31.9107
Epoch 4/50
40/40 [==============================] - 1s 27ms/step - loss: 30.3270 - val_loss: 28.8406
Epoch 5/50
40/40 [==============================] - 1s 22ms/step - loss: 27.6844 - val_loss: 26.6240
Epoch 6/50
40/40 [==============================] - 1s 23ms/step - loss: 25.8411 - val_loss: 25.0436
Epoch 7/50
40/40 [==============================] - 1s 27ms/step - loss: 24.4823 - val_loss: 23.8034
Epoch 8/50
40/40 [==============================] - 1s 18ms/step - loss: 23.3020 - val_loss: 22.7263
Epoch 9/50
40/40 [==============================] - 1s 20ms/step - loss: 22.2184 - val_loss: 21.6944
Epoch 10/50
40/40 [==============================] - 1s 20ms/step - loss: 21.1721 - val_los

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['FVI', 'DCR', 'JWO', 'XKI', 'FNK', 'XIM', 'NON', 'JCE', 'KTZ', 'DFU']
['FVI', 'DCR', 'JWO', 'XKI', 'FNK', 'XIM', 'NON', 'JCE', 'KTZ', 'DFU']
accuracy = 0.999
